In [24]:
import pandas as pd
import calendar
from time import strptime

In [10]:
main_bookings_df = pd.read_csv('hotel_bookings.csv')
len(main_bookings_df)

119390

In [7]:
main_bookings_df.isna().sum()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

In [92]:
# to get a rough estimate of room bookings, im going to assume there are only triple rooms and adults+children (not babies) are considered for total people
approx_bookings = lambda total_people: total_people//3+ (0 if total_people%3==0 else 1)

analysis_cpy_df = main_bookings_df.copy()
analysis_cpy_df['arrival_date'] = pd.to_datetime(analysis_cpy_df.apply(lambda x:f"{x.arrival_date_year}-{strptime(x.arrival_date_month,'%B').tm_mon:02}-{x.arrival_date_day_of_month}", axis=1))
analysis_cpy_df['bookings'] = analysis_cpy_df.apply(lambda x: approx_bookings(x.adults+x.children), axis=1)
analysis_cpy_df['reservation_status_date_check'] = pd.to_datetime(analysis_cpy_df['arrival_date'])+pd.to_timedelta(analysis_cpy_df['stays_in_weekend_nights']+analysis_cpy_df['stays_in_week_nights'], unit='days')

In [137]:
needed_columns = ['hotel', 'arrival_date', 'adr', 'bookings', 'revenue']

#    CONDITIONS
# filter rows where adults==0
# filter out cancellations, for simplicity
# check missing values after filtering out cancellations
#  -there arent any missing values in the critical columns
# check the date continuity considering max and min arrival dates hotelwise

final_df = analysis_cpy_df[(analysis_cpy_df.is_canceled==0)&(analysis_cpy_df.adults>0)]
final_df['revenue'] = final_df['adr']*final_df['bookings'] 
final_df = final_df[needed_columns].groupby(['hotel','arrival_date']).sum().reset_index()

print(final_df.columns)
print(final_df.hotel.unique())

for h_type in final_df.hotel.unique():
    print(h_type)
    min_arrival_dt = final_df[final_df.hotel==h_type].arrival_date.min()
    max_arrival_dt = final_df[final_df.hotel==h_type].arrival_date.max()
    print('\tactual num of records:', len(final_df[final_df.hotel==h_type]))
    print('\tneeded num of records:', len(pd.date_range(start=min_arrival_dt, end=max_arrival_dt, freq='1D')))

final_df = final_df[final_df.hotel=='Resort Hotel'].drop('hotel',axis=1).reset_index(drop=True)

Index(['hotel', 'arrival_date', 'adr', 'bookings', 'revenue'], dtype='object')
['City Hotel' 'Resort Hotel']
City Hotel
	actual num of records: 788
	needed num of records: 793
Resort Hotel
	actual num of records: 793
	needed num of records: 793


C:\Users\sanja\AppData\Local\Temp\ipykernel_10284\3573749260.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['revenue'] = final_df['adr']*final_df['bookings']


In [143]:
# min_arrival_dt = final_df.arrival_date.min()
# max_arrival_dt = final_df.arrival_date.max()
# check_dates = pd.DataFrame(pd.date_range(start=min_arrival_dt, end=max_arrival_dt, freq='1D')).rename(columns={0:'arrival_date'})
# check_dates['dummy_col'] =0